In [5]:
import json
import numpy as np
import pandas as pd
from collections import Counter

In [6]:
pip install spacy

In [7]:
!python -m spacy download en_core_web_sm

2023-11-20 15:14:34.078257: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 15:14:34.078331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 15:14:34.078380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 15:14:34.094810: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 15:14:36.109936: W tensorflow/compiler/

In [8]:
!python -m spacy download fr_core_news_sm


2023-11-20 15:14:52.362326: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 15:14:52.362389: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 15:14:52.362428: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 15:14:52.373997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 15:14:54.482015: W tensorflow/compiler/

In [9]:
import spacy
nlp_en = spacy.load("en_core_web_sm")

In [10]:
with open("trial-v1.json", "r") as f:
    trial_data = json.load(f)

df = pd.json_normalize(trial_data)

#df.head(79)

In [11]:
ar = nlp_en.get_pipe('attribute_ruler')

ar.add([[{"TEXT":"'ve"}]],{"LEMMA":"have"})
ar.add([[{"TEXT":"'m"}]],{"LEMMA":"be"})
ar.add([[{"TEXT":"n't"}]],{"LEMMA":"not"})
ar.add([[{"TEXT":"'ll"}]],{"LEMMA":"will"})
ar.add([[{"TEXT":"'re"}]],{"LEMMA":"be"})

In [24]:
pos_entities_hyp_list = []
pos_entities_tgt_list = []

for i in range(0, len(df)):

  df.at[i, 'hyp'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['hyp']) if not token.is_punct)
  df.at[i, 'tgt'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['tgt']) if not token.is_punct)

  hyp = nlp_en(df.at[i, 'hyp'])
  tgt = nlp_en(df.at[i, 'tgt'])

  pos_entities_tgt = []
  pos_entities_hyp = []

  for token in hyp:
    pos_entities_hyp.append(token.pos_ if not token.ent_type_ else token.ent_type_)

  pos_entities_hyp_list.append(pos_entities_hyp)

  for token in tgt:
    pos_entities_tgt.append(token.pos_ if not token.ent_type_ else token.ent_type_)

  pos_entities_tgt_list.append(pos_entities_tgt)

  #Feature Extraction

  hyp_tokens = [token.text for token in hyp]
  tgt_tokens = [token.text for token in tgt]

  # Find the mutual words
  intersection = dict(Counter(hyp_tokens) & Counter(tgt_tokens))
  mutual_words = list(intersection.keys())

  df.at[i, 'num_of_common'] = len(mutual_words)

  df.at[i, 'word_share_ratio'] = (len(mutual_words))/(len(hyp)+len(tgt))



df['pos_entities_hyp'] = pos_entities_hyp_list
df['pos_entities_tgt'] = pos_entities_tgt_list

In [25]:
df.head(79)

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination),hyp_len,tgt_len,num_of_common,word_share_ratio,pos_entities_hyp,pos_entities_tgt
0,a district of Kowloon China,tgt,The City <define> Chiuchow </define> is Kowloo...,the Chaoshan region where the Teochew dialect ...,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,5,9,0,0.000000,"[DET, NOUN, ADP, LOC, LOC]","[DET, GPE, NOUN, SCONJ, DET, PERSON, NOUN, AUX..."
1,wicca a witchdoctor,tgt,"“ I went to see the witchdoctor , the <define>...",South Africa a traditional tribal diviner or s...,,DM,"[Hallucination, Hallucination, Not Hallucination]",Hallucination,0.666667,3,9,1,0.083333,"[VERB, DET, NOUN]","[GPE, GPE, DET, ADJ, ADJ, NOUN, CCONJ, ADJ, NOUN]"
2,nautical a halyard,tgt,"Haul away , keeping strain on both parts of th...",nautical a staff that carry a flag or pennant ...,,DM,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.333333,3,15,2,0.111111,"[ADJ, DET, NOUN]","[ADJ, DET, NOUN, PRON, VERB, DET, NOUN, CCONJ,..."
3,the quality of be rational,tgt,And what difference is there between the credi...,archaic the quality or state of be rational,,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000,5,8,5,0.384615,"[DET, NOUN, ADP, AUX, ADJ]","[VERB, DET, NOUN, CCONJ, NOUN, ADP, AUX, ADJ]"
4,uncountable the study of tree,tgt,It is now generally supposed that the forbidde...,the worship of tree,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,5,4,3,0.333333,"[ADJ, DET, NOUN, ADP, NOUN]","[DET, NOUN, ADP, NOUN]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,I have get the floor and the furniture,either,J'ai poli le plancher et les meubles.,I polish up the floor and furniture,,MT,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,8,7,5,0.333333,"[PRON, AUX, VERB, DET, NOUN, CCONJ, DET, NOUN]","[PRON, VERB, ADP, DET, NOUN, CCONJ, NOUN]"
75,I can not do it alone,either,Je ne peux le faire seule. Il vous faut m'aider.,I can not do it alone you have to help I,,MT,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.333333,6,11,6,0.352941,"[PRON, AUX, PART, VERB, PRON, ADV]","[PRON, AUX, PART, VERB, PRON, ADJ, PRON, VERB,..."
76,he lose what he receive from his father,either,Il a perdu ce qu'il a reçu de son père.,he waste that which he have receive from his f...,,MT,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000,8,10,5,0.277778,"[PRON, VERB, PRON, PRON, VERB, ADP, PRON, NOUN]","[PRON, VERB, PRON, PRON, PRON, AUX, VERB, ADP,..."
77,we have discover a lot of news,either,Nous avons découvert une grande nouveauté.,we have discover a great novelty,,MT,"[Hallucination, Not Hallucination, Hallucination]",Hallucination,0.666667,7,6,4,0.307692,"[PRON, AUX, VERB, DET, NOUN, ADP, NOUN]","[PRON, AUX, VERB, DET, ADJ, NOUN]"


In [14]:
df.to_json(r'processed2_df.json', orient='records')